## Comparing felsenstein with arDCA

In [1]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles, PottsGauge
using AncestralSequenceReconstruction # core package
using JLD2 # used to load ArDCA models
using TreeTools # to handle p
using ArDCA, KitMSA

nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];

@load "../data_Genie/pars_dbd.jld2"; 

J_rs = permutedims(J_dbd,[1,3,2,4]);
J_zsg, h_zsg = gauge(permutedims(J_dbd, [1,3,2,4]), h_dbd, ZeroSumGauge());
PottsGauge.testgauge(J_zsg,h_zsg,J_rs,h_dbd);
J = permutedims(J_zsg, [1,3,2,4]); h = deepcopy(h_zsg);

#@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;
@load "../data_Anc/3_start_seq_and_sweeps4DBD_ASR.jld2"

q = 21; L =76;

@load "../data_Genie/3_seqs_sim.jld2";
@load "../data_Genie/cie_dbd.jld2";
sweeps_sim = res_all[1].steps ./ L;

hs = [0 .* h, h, h]; Js = [J, J, 0 .*J] ; names = ["Only couplings", "Standard", "Profile"];
alphas = deepcopy(names);
mus1 = [1., 10., 50., 100., 200.];
#alphas = [0., 0.2, 0.5, 0.7, 1.];


eq_samples = [];

for i in 1:length(names)
    if i == 2
        push!(eq_samples, nat_msa)
    else
        #file_seqs = "../eq_samples_alpha_for_arDCA/sample_alpha$(alphas[i]).fa"
        file_seqs = "../eq_sample_for_arDCA/sample_$(alphas[i]).fa"
        push!(eq_samples, read_fasta_alignment(file_seqs,0.9))
    end
end


start_seq = [Int.(eq_samples[i][:,1]) for i in 1:length(alphas)];

ens_start = [PhyloTools.energy(start_seq[i], hs[i], Js[i]) for i in 1:length(alphas)];



tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
leaves_msas = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
hams_leaves =  zeros(length(alphas), length(mus1));
hams_intra_leaves = zeros(length(alphas), length(mus1));

num = 1;
#generating data and reading it
for alpha in names
    for n in 1:length(mus1) 
        @time res1 = run_evolution_ontree(start_seq[num], tree_file, 
        hs[num], Js[num], mu = mus1[n], p = 0.5);
        file_seqs = "../res_algos/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
        leavestofasta(file_seqs, res1) 
        leaves_msas[num,n] = Int.(read_fasta_alignment(file_seqs,0.9))
        hams_leaves[num,n] = mean(ham_dist_nogap(start_seq[num], leaves_msas[num,n]))
        hams_intra_leaves[num,n] = pairwise_ham_dist(leaves_msas[num,n], n_seq = 300)
    end
    num += 1
end


## doing felsenstein inference
res_ASR = zeros(Int, length(alphas), L, length(mus1));
pp = Matrix{Matrix{Float64}}(undef, length(alphas), length(mus1));
rec_msas = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
ens = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
hams = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
ens_ML =  zeros(length(alphas), length(mus1));
hams_ML =  zeros(length(alphas), length(mus1));


  
#inference with felsenstein

num = 1; for alpha in names 
    for n in 1:length(mus1)
        file_seqs = "../res_algos/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
        mu = infer_mu(tree_file, file_seqs)
        res, p = Felsenstein2(file_seqs, tree_file, file_seqs, mu)
        res_ASR[num,:,n] .= Int.(res)
        pp[num,n] = p
        rec_msas[num,n] = sampling_ANC(pp[num,n], n_seq = 500)
        hams[num,n] = ham_dist_nogap(start_seq[num], rec_msas[num,n])
        ens[num,n] = PhyloTools.energy(rec_msas[num,n],  hs[num],  Js[num]) .- ens_start[num]
        hams_ML[num,n] = ham_dist_nogap(start_seq[num], res_ASR[num,:,n])
        ens_ML[num,n] = PhyloTools.energy(res_ASR[num,:,n], hs[num],  Js[num]) - ens_start[num]
    end
    num +=1
end



#learning arDCA models on equilibrium data

ar_models = []
for i in 1:length(names)
    if i == 2
        file_equil_seqs = "../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m"
        arnet,arvar=ardca(file_equil_seqs);
        push!(ar_models, AutoRegressiveModel(arnet))
    else
        file_equil_seqs = "../eq_sample_for_arDCA/sample_$(names[i]).fa"
        arnet,arvar=ardca(file_equil_seqs);
        push!(ar_models, AutoRegressiveModel(arnet))
    end
end

#inference with arDCA

res_ASR_arDCA = zeros(Int, length(alphas), L, length(mus1));
rec_msas_arDCA = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
ens_arDCA = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
hams_arDCA = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
ens_ML_arDCA =  zeros(length(alphas), length(mus1));
hams_ML_arDCA =  zeros(length(alphas), length(mus1));

num = 1; for alpha in names
@time for n in 1:length(mus1)
        fasta_file = "../res_algos/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
    
        strategy = ASRMethod(;joint = false, # (default) - joint reconstruction not functional yet
            ML = true, # (default)
            verbosity = 1, # the default is 0. 
            optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
            optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
            repetitions = 1, # (default) - for Bayesian reconstruction, multiple repetitions of the reconstruction process can be done to sample likely ancestors
            );
        opt_tree, reconstructed_sequences = infer_ancestral(
            tree_file, fasta_file, ar_models[num], strategy
            );
        res_ASR_arDCA[num,:,n] = KitMSA.string2vec(reconstructed_sequences["NODE_1"]);
        node_list=["NODE_1"];
        
        
        #this commented part is for the bayesian arDCA method
        strategy_bayesian = ASRMethod(;
	joint = false, # (default) - joint reconstruction not functional yet
	ML = false, # (default)
	verbosity = 1, # the default is 0. 
	optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
	optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
	repetitions = 100,

)
        
    file_out = "../res_algos/ardca_inf/alpha$(alpha)_ardca_mu$(round(mus1[n], 
        digits = 2)).fa";
        infer_ancestral(
               tree_file, fasta_file, ar_models[num], strategy_bayesian;
               alignment_per_node=true, node_list = ["NODE_1"], outfasta = file_out);
    println(n)
    end 
    num += 1
end

num = 1
for alpha in alphas
    for n in 1:length(mus1)
        
        hams_ML_arDCA[num,n] = ham_dist_nogap(start_seq[num], res_ASR_arDCA[num,:,n])
        ens_ML_arDCA[num,n] = PhyloTools.energy(res_ASR_arDCA[num,:,n], hs[num], Js[num]) - ens_start[num] 
    
        rec_msas_arDCA[num,n] = read_fasta_alignment("../res_algos/ardca_inf/alpha$(alpha)_ardca_mu$(round(mus1[n], digits = 2))_NODE_1.fa", 0.9)
        
        hams_arDCA[num,n] = ham_dist_nogap(start_seq[num], Int.(rec_msas_arDCA[num,n]))
        ens_arDCA[num,n] = PhyloTools.energy(rec_msas_arDCA[num,n], hs[num], Js[num]) .- ens_start[num]
    end
    num += 1
end


close("all")
fig, axs = plt.subplots(1, length(alphas), figsize = (30,5) )
num = 1; for alpha in alphas
    axs[num].errorbar(hams_leaves[num,:] ./ L, [mean(hams[num,n])./L for n in 1:length(mus1)] ,
    yerr = [std(hams[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs[num].scatter(hams_leaves[num,:] ./ L, hams_ML[num,:]./L, marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs[num].errorbar(hams_leaves[num,:] ./ L, [mean(hams_arDCA[num,n])./L for n in 1:length(mus1)],
    yerr = [std(hams_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs[num].scatter(hams_leaves[num,:] ./ L, hams_ML_arDCA[num,:]./L, marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs[num].set_title("Model = $(alpha)", fontsize = 20)
    num+=1
end

fig.supxlabel("Leaves-root hamming", fontsize = 20)
axs[1].legend()
axs[1].set_ylabel("Root-reconstruction \n hamming", fontsize = 20)
savefig("../alpha_ham_ASR_felse_vs_arDCA.png")


close("all")
fig, axs = plt.subplots(1, length(alphas), figsize = (30,5) )
num = 1; for alpha in alphas
    axs[num].errorbar(hams_intra_leaves[num,:] ./ L, [mean(hams[num,n])./L for n in 1:length(mus1)] ,
    yerr = [std(hams[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs[num].scatter(hams_intra_leaves[num,:] ./ L, hams_ML[num,:]./L, marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs[num].errorbar(hams_intra_leaves[num,:] ./ L, [mean(hams_arDCA[num,n])./L for n in 1:length(mus1)],
    yerr = [std(hams_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs[num].scatter(hams_intra_leaves[num,:] ./ L, hams_ML_arDCA[num,:]./L, marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs[num].set_title("Model = $(alpha)", fontsize = 20)
    num+=1
end
fig.supxlabel("Leaves pairwise hamming", fontsize = 20)
axs[1].legend()
axs[1].set_ylabel("Root-reconstruction \n hamming", fontsize = 20)
savefig("../alpha_intra_ham_ASR_felse_vs_arDCA.png")


close("all")
fig, axs = plt.subplots(1, length(alphas), figsize = (30,5) )
num = 1; for alpha in alphas
    axs[num].errorbar(hams_leaves[num,:] ./ L, [mean(ens[num,n]) for n in 1:length(mus1)] ,
    yerr = [std(ens[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs[num].scatter(hams_leaves[num,:] ./ L, ens_ML[num,:], marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs[num].errorbar(hams_leaves[num,:] ./ L, [mean(ens_arDCA[num,n]) for n in 1:length(mus1)],
    yerr = [std(ens_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs[num].scatter(hams_leaves[num,:] ./ L, ens_ML_arDCA[num,:], marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs[num].set_title("Model = $(alpha)", fontsize = 20)
    num+=1
end

fig.supxlabel("Leaves-root hamming", fontsize = 20)
axs[1].legend()
axs[1].set_ylabel("E_ASR - E_wt", fontsize = 20)
savefig("../alpha_en_ASR_felse_vs_arDCA.png")


close("all")
fig, axs = plt.subplots(1, length(alphas), figsize = (30,5) )
num = 1; for alpha in alphas
    axs[num].errorbar(hams_intra_leaves[num,:] ./ L, [mean(ens[num,n]) for n in 1:length(mus1)] ,
    yerr = [std(ens[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs[num].scatter(hams_intra_leaves[num,:] ./ L, ens_ML[num,:], marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs[num].errorbar(hams_intra_leaves[num,:] ./ L, [mean(ens_arDCA[num,n]) for n in 1:length(mus1)],
    yerr = [std(ens_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs[num].scatter(hams_intra_leaves[num,:] ./ L, ens_ML_arDCA[num,:] , marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs[num].set_title("Model = $(alpha)", fontsize = 20)
    num+=1
end
fig.supxlabel("Leaves pairwise hamming", fontsize = 20)
axs[1].legend()
axs[1].set_ylabel("E_ASR - E_wt", fontsize = 20)
savefig("../alpha_intra_en_ASR_felse_vs_arDCA.png")

### plotting as a func of alpha

close("all")
fig, axs = plt.subplots(1, length(mus1), figsize = (30,5) )
for num in 1:length(mus1)
    axs[num].errorbar(alphas, [(mean(hams[n,num]) .- mean(hams_arDCA[n,num])) ./L for n in 1:length(alphas)] ,
    yerr = [(std(hams[n,num]) .- std(hams_arDCA[n,num]))./L for n in 1:length(alphas)], 
    color = "blue", label = "Bayes"); 
    axs[num].scatter(alphas, (hams_ML[:,num] .- hams_ML_arDCA[:,num]) ./L, marker = "X", color = "blue", s = 300, label = "ML");
    axs[num].plot(alphas, [0 for i in 1:length(alphas)], color = "grey")
end
fig.supxlabel("Models", fontsize = 20)
axs[1].legend()
axs[1].set_ylabel("Root-reconstruction \n hamming (felse-arDCA)", fontsize = 20)
savefig("../1.png")

close("all")
fig, axs = plt.subplots(1, length(mus1), figsize = (30,5) )
for num in 1:length(mus1)
    axs[num].plot(alphas, [(mean(ens[n,num]) .- mean(ens_arDCA[n,num])) ./L for n in 1:length(alphas)] ,
    color = "blue", label = "Bayes"); 
    axs[num].scatter(alphas, ens_ML[:,num] .- ens_ML_arDCA[:,num], marker = "X", color = "blue", s = 300, label = "ML");
    axs[num].plot(alphas, [0 for i in 1:length(alphas)], color = "grey")
end
fig.supxlabel("Models", fontsize = 20)
axs[1].legend()
axs[1].set_ylabel("E_ASR - E_wt (felse-arDCA)", fontsize = 20)
savefig("../2.png")




LoadError: SystemError: ../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m: No such file or directory

## single trial

In [ ]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles, PottsGauge
using AncestralSequenceReconstruction # core package
using JLD2 # used to load ArDCA models
using TreeTools # to handle p
using ArDCA, KitMSA
using Distributions


@load "../data_Genie/pars_dbd.jld2"; 

h = h_dbd; J = J_dbd;



q = 21; L =76; 
hs = [h, h, h]; Js = [J, J, 0 .*J] ; names = ["Standard", "Standard", "Profile"];
alphas = deepcopy(names); alpha = "Only_couplings"; 
# mus1 = [1., 5., 10., 50., 100.];


#tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
tree_file = "../res_trees/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301_rescaled_depth_16.nwk"
mus1 = 0.33 .* [1., 10., 30.,  50., 100.];


#file_seqs = "../potts_alignment_only_pairwise.fa"; 
file_seqs = "../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m"; 
eq_samples = [read_fasta_alignment(file_seqs,0.9) for _ in 1:3];


start_seq = [Int.(eq_samples[i][:,2]) for i in 1:length(alphas)];

ens_start = [PhyloTools.energy(start_seq[i], hs[i], Js[i]) for i in 1:length(alphas)];

leaves_msas = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
hams_leaves =  zeros(length(alphas), length(mus1));
hams_intra_leaves = zeros(length(alphas), length(mus1));

num = 1;  
#generating data and reading it
#for alpha in names
    for n in 1:length(mus1) 
        @time res1 = run_evolution_ontree(start_seq[num], tree_file, 
        hs[num], Js[num], mu = mus1[n], temp = 1., p = 0.5);
        file_seqs = "../res_last_tree/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
        leavestofasta(file_seqs, res1) 
        leaves_msas[num,n] = Int.(read_fasta_alignment(file_seqs,0.9))
        hams_leaves[num,n] = mean(ham_dist_nogap(start_seq[num], leaves_msas[num,n]))
        hams_intra_leaves[num,n] = pairwise_ham_dist(leaves_msas[num,n], n_seq = 300)
    end
    #num += 1
#end

hams_leaves


## doing felsenstein inference
res_ASR = zeros(Int, length(alphas), L, length(mus1));
pp = Matrix{Matrix{Float64}}(undef, length(alphas), length(mus1));
rec_msas = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
ens = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
hams = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
ens_ML =  zeros(length(alphas), length(mus1));
hams_ML =  zeros(length(alphas), length(mus1));


  
#inference with felsenstein

num = 1; #for alpha in names 
    for n in 1:length(mus1)
        file_seqs = "../res_last_tree/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
        mu = infer_mu(tree_file, file_seqs)
        res, p = Felsenstein2(file_seqs, tree_file, file_seqs, mu)
        res_ASR[num,:,n] .= Int.(res)
        pp[num,n] = p
        rec_msas[num,n] = sampling_ANC(pp[num,n], n_seq = 500)
        hams[num,n] = ham_dist_nogap(start_seq[num], rec_msas[num,n])
        ens[num,n] = PhyloTools.energy(rec_msas[num,n],  hs[num],  Js[num]) .- ens_start[num]
        hams_ML[num,n] = ham_dist_nogap(start_seq[num], res_ASR[num,:,n])
        ens_ML[num,n] = PhyloTools.energy(res_ASR[num,:,n], hs[num],  Js[num]) - ens_start[num]
    end
    #num +=1
#end



#learning arDCA models on equilibrium data

ar_models = []
for i in 1:length(names)
    #file_equil_seqs = "../potts_alignment_only_pairwise.fa"
    
    file_equil_seqs = "../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m";
    arnet,arvar=ardca(file_equil_seqs);
    push!(ar_models, AutoRegressiveModel(arnet))
end

#inference with arDCA

res_ASR_arDCA = zeros(Int, length(alphas), L, length(mus1));
rec_msas_arDCA = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
ens_arDCA = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
hams_arDCA = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
ens_ML_arDCA =  zeros(length(alphas), length(mus1));
hams_ML_arDCA =  zeros(length(alphas), length(mus1));

num = 1; #for alpha in names
@time for n in 1:length(mus1)
        fasta_file = "../res_last_tree/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
    
        strategy = ASRMethod(;joint = false, # (default) - joint reconstruction not functional yet
            ML = true, # (default)
            verbosity = 1, # the default is 0. 
            optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
            optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
            repetitions = 1, # (default) - for Bayesian reconstruction, multiple repetitions of the reconstruction process can be done to sample likely ancestors
            );
        opt_tree, reconstructed_sequences = infer_ancestral(
            tree_file, fasta_file, ar_models[num], strategy
            );
        res_ASR_arDCA[num,:,n] = KitMSA.string2vec(reconstructed_sequences["NODE_1"]);
        node_list=["NODE_1"];
        
        
        #this commented part is for the bayesian arDCA method
        strategy_bayesian = ASRMethod(;
	joint = false, # (default) - joint reconstruction not functional yet
	ML = false, # (default)
	verbosity = 1, # the default is 0. 
	optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
	optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
	repetitions = 100,

)
        
    file_out = "../res_last_tree/ardca_inf/alpha$(alpha)_ardca_mu$(round(mus1[n], 
        digits = 2)).fa";
        infer_ancestral(
               tree_file, fasta_file, ar_models[num], strategy_bayesian;
               alignment_per_node=true, node_list = ["NODE_1"], outfasta = file_out);
    println(n)
    end 
    #num += 1
#end

num = 1
#for alpha in alphas
    for n in 1:length(mus1)
        
        hams_ML_arDCA[num,n] = ham_dist_nogap(start_seq[num], res_ASR_arDCA[num,:,n])
        ens_ML_arDCA[num,n] = PhyloTools.energy(res_ASR_arDCA[num,:,n], hs[num], Js[num]) - ens_start[num] 
    
        rec_msas_arDCA[num,n] = read_fasta_alignment("../res_last_tree/ardca_inf/alpha$(alpha)_ardca_mu$(round(mus1[n], digits = 2))_NODE_1.fa", 0.9)
        
        hams_arDCA[num,n] = ham_dist_nogap(start_seq[num], Int.(rec_msas_arDCA[num,n]))
        ens_arDCA[num,n] = PhyloTools.energy(rec_msas_arDCA[num,n], hs[num], Js[num]) .- ens_start[num]
    end
    #num += 1
#end


close("all")
fig, axs = plt.subplots(figsize = (8,4) )
num = 1; #for alpha in alphas
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(hams[num,n])./L for n in 1:length(mus1)] ,
    yerr = [std(hams[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, hams_ML[num,:]./L, marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(hams_arDCA[num,n])./L for n in 1:length(mus1)],
    yerr = [std(hams_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, hams_ML_arDCA[num,:]./L, marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs.set_title("Model = $(alpha)", fontsize = 20)
    #num+=1
#end

fig.supxlabel("Leaves-root hamming", fontsize = 20)
axs.legend()
axs.set_ylabel("Root-reconstruction \n hamming", fontsize = 20)
savefig("../single_alpha_ham_ASR_felse_vs_arDCA.png")


close("all")
fig, axs = plt.subplots(figsize = (8,4) )
num = 1; #for alpha in alphas
    axs.errorbar(hams_intra_leaves[num,:] ./ L, [mean(hams[num,n])./L for n in 1:length(mus1)] ,
    yerr = [std(hams[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs.scatter(hams_intra_leaves[num,:] ./ L, hams_ML[num,:]./L, marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs.errorbar(hams_intra_leaves[num,:] ./ L, [mean(hams_arDCA[num,n])./L for n in 1:length(mus1)],
    yerr = [std(hams_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs.scatter(hams_intra_leaves[num,:] ./ L, hams_ML_arDCA[num,:]./L, marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs.set_title("Model = $(alpha)", fontsize = 20)
    #num+=1
#end
fig.supxlabel("Leaves pairwise hamming", fontsize = 20)
axs.legend()
axs.set_ylabel("Root-reconstruction \n hamming", fontsize = 20)
savefig("../single_alpha_intra_ham_ASR_felse_vs_arDCA.png")


close("all")
fig, axs = plt.subplots(figsize = (8,4) )
num = 1; #for alpha in alphas
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(ens[num,n]) for n in 1:length(mus1)] ,
    yerr = [std(ens[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, ens_ML[num,:], marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(ens_arDCA[num,n]) for n in 1:length(mus1)],
    yerr = [std(ens_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, ens_ML_arDCA[num,:], marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs.set_title("Model = $(alpha)", fontsize = 20)
    #num+=1
#end

fig.supxlabel("Leaves-root hamming", fontsize = 20)
axs.legend()
axs.set_ylabel("E_ASR - E_wt", fontsize = 20)
savefig("../single_alpha_en_ASR_felse_vs_arDCA.png")


close("all")
fig, axs = plt.subplots(figsize = (8,4) )
num = 1; #for alpha in alphas
    axs.errorbar(hams_intra_leaves[num,:] ./ L, [mean(ens[num,n]) for n in 1:length(mus1)] ,
    yerr = [std(ens[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs.scatter(hams_intra_leaves[num,:] ./ L, ens_ML[num,:], marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs.errorbar(hams_intra_leaves[num,:] ./ L, [mean(ens_arDCA[num,n]) for n in 1:length(mus1)],
    yerr = [std(ens_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs.scatter(hams_intra_leaves[num,:] ./ L, ens_ML_arDCA[num,:] , marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs.set_title("Model = $(alpha)", fontsize = 20)
    #num+=1
#end
fig.supxlabel("Leaves pairwise hamming", fontsize = 20)
axs.legend()
axs.set_ylabel("E_ASR - E_wt", fontsize = 20)
savefig("../single_alpha_intra_en_ASR_felse_vs_arDCA.png")



## Analysing fields and coupling importance and generating eq samples for arDCA

In [ ]:
using Genie, DCAUtils, JLD2, PyPlot, Statistics, PottsGauge

#@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;

@load "../data_Genie/pars_dbd.jld2"; 

J_rs = permutedims(J_dbd,[1,3,2,4]);
J_zsg, h_zsg = gauge(permutedims(J_dbd, [1,3,2,4]), h_dbd, ZeroSumGauge());
PottsGauge.testgauge(J_zsg,h_zsg,J_rs,h_dbd);
J = permutedims(J_zsg, [1,3,2,4]); h = deepcopy(h_zsg);



@load "../data_Anc/3_start_seq_and_sweeps4DBD_ASR.jld2"

nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
ens_nat_h = mean(energy(nat_msa, h, 0 .*J))
ens_nat_J = mean(energy(nat_msa, 0 * h, J))
ens_nat = mean(energy(nat_msa, h, J)) 

sigm(alpha::Float64) = 1/(1+exp(-alpha))
sigm(alpha::Vector{Float64}) = 1 ./ ( 1 .+ exp.( .- alpha))

function temp(alphas::Vector{Float64}, x, y)
    return 2 .*(x .* alphas .+ y .* (1 .- alphas))./(x+y)
end

function temp(alpha::Float64, x, y)
    return 2 *(x * alpha + y * (1 - alpha))/(x+y)
end

function field_importance(alphas::Vector{Float64}, x, y)
    return ((alphas .* x) .^2) ./ ((alphas .* x).^2 .+ ((1 .- alphas) .* y).^2 )
end

function field_importance(alpha::Float64, x, y)
    return ((alpha * x) ^2) / ((alpha * x) + (1 - alpha) *y)^2
end

alphas = [0., 0.2, 0.5, 0.7, 1.];
#alphas = [0.01*i for i in 1:100];
sigm_alphas = sigm(alphas);
temps = temp(alphas, ens_nat_h, ens_nat_J)
fields_perc = field_importance(alphas, ens_nat_h, ens_nat_J)

close("all"); plt.plot(alphas, fields_perc); plt.xlabel("Alpha"); plt.ylabel("Energy % due to fields"); savefig("../ciao.png")
close("all"); plt.plot(alphas, temps); savefig("../ciao1.png")     
        
ens_modified = zeros(length(alphas), size(nat_msa,2));

for i in 1:length(alphas)
    ens_modified[i,:] = energy(nat_msa, 2*alphas[i]*h, 2 .* (1-alphas[i]) .* J)
end

close("all"); plt.errorbar(alphas, mean(ens_modified, dims = 2)[:] ./ temps, yerr = std(ens_modified ./temps, dims = 2)[:]
    ); plt.xlabel("Alpha"); plt.ylabel("Mean nat energy/T"); savefig("../alpha_energy_nat.png")






q = 21; L =76;

start_seq = start_msa[:,1]

#start_msa = hcat([start_seq for i in 1:5000]...);
#start_msa = nat_msa[:,1:5000];
start_msa = Int8.(rand(1:21,L,5000));

L = length(start_seq);

#alphas = [0., 0.2, 0.5, 0.7, 1.]; MCMC_steps = [10^5, 10^5, 3*10^4, 10^4, 10^4];
#MCMC_steps = [10^5, 10^5, 10^5];
hs = [0 .* h, h, h]; Js = [J, J, 0 .*J] ; MCMC_steps = [8*10^4, 10^3, 8*10^4
    ]; each_steps = [1000, 200,1000]; names = ["Only couplings", "Standard", "Profile"];
mean_dist = [[] for i in 1:length(alphas)];
mean_intra_dist = [[] for i in 1:length(alphas)];
mean_t2_dist = [[] for i in 1:length(alphas)];
mean_en = [[] for i in 1:length(alphas)];
steps = [[] for i in 1:length(alphas)];
steps_t2 = [[] for i in 1:length(alphas)];
res_all = [];
for i in 1:length(hs)
    @time res = run_evolution(start_msa, 
    hs[i], 
    Js[i],
    p = 0.5, 
    temp = 1., 
    N_steps = MCMC_steps[i],  
    each_step = 500, 
    verbose = false);
    push!(res_all, res)
    for n in 1:length(res_all[i].steps)
        push!(mean_dist[i], mean(ham_dist(start_msa, res.step_msa[n])))
        push!(mean_intra_dist[i], pairwise_ham_dist(res.step_msa[n], n_seq = 500))
        push!(mean_en[i], mean(energy(res.step_msa[n], hs[i], Js[i])))
        push!(steps[i], res.steps[n])
        if length(findall(x -> x == ((res.steps[n]-1)/2)+1, res.steps)) == 1
            push!(steps_t2[i], res.steps[n])
            ind = findall(x -> x == ((res.steps[n]-1)/2)+1, res.steps)[1]
            #println(ind)
            push!(mean_t2_dist[i], mean(Genie.ham_dist(res.step_msa[n], res.step_msa[ind]))) 
        end
    end
end

close("all"); for i in 1:length(hs)
    plt.plot(steps[i] ./ L, mean_dist[i] ./ L, label = "$(names[i])")
end; plt.xscale("log");  plt.xlabel("Sweeps"); plt.ylabel("Hamming_from_start"
    ); plt.legend();savefig("../ciao_dist.png")

close("all"); for i in 1:length(hs)
    plt.plot(steps[i] ./ L, mean_intra_dist[i] ./ L, label = "$(names[i])")
    plt.scatter(steps_t2[i] ./ L, mean_t2_dist[i] ./ L, label = "$(names[i])")
end; plt.xscale("log");  plt.xlabel("Sweeps"); plt.ylabel("Pairwise Hamming"
    ); plt.legend();savefig("../ciao_intra_dist.png")

close("all"); for i in 1:length(hs)
    plt.plot(steps[i] ./ L, mean_en[i] , label = "$(names[i])")
end; plt.xscale("log"); plt.xlabel("Sweeps"); plt.legend();savefig("../ciao_en.png")

    
    
    
using KitMSA
for i in 1:length(names)
    #file_seqs = "../eq_samples_alpha_for_arDCA/sample_alpha$(alphas[i]).fa"
    file_seqs = "../eq_sample_for_arDCA/sample_$(names[i]).fa"
    matrix2fasta(file_seqs, res_all[i].step_msa[end]')
end
    
    
#other part

using Revise, PhyloTools, DCAUtils, JLD2, PyPlot, Statistics
using Distributions, Random

L = 20; q = 21; M = 1000;
μ = 0.0;      
b = sqrt(0.5*0.2);     
laplace_dist = Laplace(μ, b);

start_msa = rand(1:q, L, M);
J_diag = zeros(q,L,q,L);
h_diag = zeros(q,L);

for i in 1:L
    for j in i+1:L
        value = rand(laplace_dist)
        for a in 1:q
            J_diag[a,i,a,j] = 2. #value
            J_diag[a,j,a,i] = J_diag[a,i,a,j] 
        end
    end
end

@time new_msa = PhyloTools.reshuffle_entr(start_msa, 0. .* h_diag, 1 .* J_diag, 
    times = 1.);

f1_0, f2_0 = compute_weighted_frequencies(Int8.(start_msa),22, 0.); c_0 = reshape(f2_0 - f1_0*f1_0', q, L, q, L);
f1, f2 = compute_weighted_frequencies(Int8.(new_msa),22, 0.); c= reshape(f2 - f1*f1', q, L, q, L);

indices = partialsortperm(J_diag[:], 1:1000, rev=true)

close("all"); plt.scatter(c_0[indices], c[indices]);plt.plot(
    [minimum(c_0[indices]), maximum(c_0[indices])],[minimum(c_0[indices]), maximum(c_0[indices])]
    ); savefig("../trial.png")



[mean(c_0[:]) mean(c[:])]
[std(c_0[:]) std(c[:])]

close("all"); plt.hist(c[:],density = true, histtype="step",label = "reshuf"); plt.hist(c_0[:],
    density = true,histtype = "step",label = "random"); plt.legend();savefig("../hist.png")

#close("all"); plt.plot(steps, ens); plt.xscale("log"); savefig("../energia.png");

using Tullio


L = 76
nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9)[1:L,:];
rand_msa = rand(1:21,L,10000); 
f1_0, f2_0 = compute_weighted_frequencies(Int8.(rand_msa),22, 0.); c_0 = reshape(f2_0 - f1_0*f1_0', q, L, q, L);
f1, f2 = compute_weighted_frequencies(Int8.(nat_msa),22, 0.); c= reshape(f2 - f1*f1', q, L, q, L);

[mean(c_0[:]) mean(c[:])]
[std(c_0[:]) std(c[:])]

close("all"); plt.hist(c[:],density = true, histtype="step",label = "reshuf"); plt.hist(c_0[:],
    density = true,histtype = "step",label = "random"); plt.legend();savefig("../hist.png")

close("all")
# Create a figure with two subplots
fig, ax = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns, square aspect

# Plot the first matrix
cax1 = ax[1].imshow(f2_0 - f1_0*f1_0', cmap="viridis", aspect="equal")
fig.colorbar(cax1, ax=ax[1])
ax[1].set_title("Random")

# Plot the second matrix
cax2 = ax[2].imshow(f2 - f1*f1', cmap="viridis", aspect="equal")  # Different colormap for variety
fig.colorbar(cax2, ax=ax[2])
ax[2].set_title("Nat")

savefig("../imshow.png")


indices = partialsortperm(c[:], 1:1000, rev=true)

close("all"); plt.scatter(c_0[indices], c[indices]);plt.plot(
    [minimum(c_0[indices]), maximum(c_0[indices])],[minimum(c_0[indices]), maximum(c_0[indices])]
    ); savefig("../trial.png")


## Comparing algos for different alphas

In [ ]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles
using AncestralSequenceReconstruction # core package
using JLD2 # used to load ArDCA models
using TreeTools # to handle p
using ArDCA, KitMSA

nat_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);
w = compute_weights(nat_msa, 22, 0.2)[1];
@load "../data_Genie/pars_dbd.jld2"; h = h_dbd; J = J_dbd;
@load "../data_Anc/3_start_seq_and_sweeps4DBD_ASR.jld2"

q = 21; L =76;

#=@load "../data_Genie/3_seqs_sim.jld2";
@load "../data_Genie/cie_dbd.jld2";
sweeps_sim = res_all[1].steps ./ L; =#


#@time m1 = PhyloTools.info_timescales(res_all[1].step_msa, nat_msa, h, J, res_all[1].steps);


tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
tree = read_tree(tree_file, node_data_type = Seq);
dd = [distance(tree.root, a) for a in leaves(tree)];

branch_d = mean([distance(tree.root, a) for a in leaves(tree)]);
#mus = sweeps[[1,2,3,4,6]] ./ branch_d;
mus = readdlm("../data_Anc/OFF_3_seq_DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301/mus.txt")[:]
sweeps = mus .* branch_d;


folder = "../data_Anc/OFF_3_seq_DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301"
    
sweeps_on_tree = []; distances_on_tree = []; for mu in mus 
    ts, ds = PhyloTools.extract_distances(tree_file, joinpath(folder,
            "seq1/seq1_mu$(round(mu,digits = 2)).fa"), Int.(start_msa[:,1]))
    push!(sweeps_on_tree, ts.*mu)   
    push!(distances_on_tree, ds./L)   
end



for i in 1:length(mus)
close("all");plt.plot(sweeps_sim, mean(m1.cie_t[:,varr], dims = 2), color = "blue"); plt.plot(
    sweeps_sim, mean(m1.cie_t[:,cons], dims = 2), color = "green"); plt.plot(sweeps_sim, 
    mean(m1.cie_t[:,epis], dims = 2), color = "red"); plt.scatter(sweeps_on_tree[i], 
        distances_on_tree[i], color = "grey") 
plt.xlabel("Sweeps");plt.xscale("log"); plt.title("MU = $(mus[i])"); savefig("../trial$(i).png") 
    
end


mus1 = [1., 10., 50., 100., 200.];

alpha = 100.;
idx = 2;

step_msa_1 = [];step_msa_2 = [];step_msa_3 = []; res_all1 = []; res_all2 = []; res_all3 = [];
for n in 1:length(mus1) 
    @time res1 = run_evolution_ontree(Int.(start_msa[:,idx]), tree_file, 
    h, alpha .* J, mu = mus1[n], p = 0.5); 
    push!(res_all1, res1)
    push!(step_msa_1, msa_from_leafs(res1))
end


for n in 1:length(mus1)
    leavestofasta("../res_algos/alpha$(alpha)_potts_seq$(idx)_mu$(round(mus1[n], 
        digits = 2)).fa", res_all1[n]) 
end

res_ASR = zeros(Int, 3, L, length(mus1));
pp = Matrix{Matrix{Float64}}(undef, 3, length(mus1));
rec_msas = Matrix{Matrix{Int}}(undef, 3, length(mus1));
#rec_msas_reshuf = Matrix{Matrix{Int}}(undef, 3, length(mus));
ens = Matrix{Array{Float64,1}}(undef, 3, length(mus1));
hams = Matrix{Array{Float64,1}}(undef, 3, length(mus1));
#ens_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
#hams_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
ens_ML =  zeros(3, length(mus1));
hams_ML =  zeros(3, length(mus1));

    
    for n in 1:length(mus1)
        file_seqs = "../res_algos/alpha$(alpha)_potts_seq$(idx)_mu$(round(mus1[n], 
        digits = 2)).fa"
        mu = infer_mu(tree_file, file_seqs)
        res, p = Felsenstein2(file_seqs, tree_file, file_seqs, mu)
        res_ASR[idx,:,n] .= Int.(res)
        pp[idx,n] = p
        rec_msas[idx,n] = sampling_ANC(pp[idx,n], n_seq = 500)
        hams[idx,n] = ham_dist_nogap(Int.(start_msa[:,idx]), rec_msas[idx,n])
        ens[idx,n] = PhyloTools.energy(rec_msas[idx,n], h, J)
        #@time rec_msas_reshuf[idx,n] = reshuffle_entr(rec_msas[idx,n], h, J, temp = 0.2)
        #hams_reshuf[idx,n] = ham_dist(Int.(start_msa[:,idx]), rec_msas_reshuf[idx,n])
        #ens_reshuf[idx,n] = energy(rec_msas_reshuf[idx,n], h, J)
        hams_ML[idx,n] = ham_dist_nogap(Int.(start_msa[:,idx]), res_ASR[idx,:,n])
        ens_ML[idx,n] = PhyloTools.energy(res_ASR[idx,:,n], h, J)
    end


nat_file = "../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m";
arnet,arvar=ardca(nat_file);ar_model = AutoRegressiveModel(arnet);

res_ASR_arDCA = zeros(Int, 3, L, length(mus1));


    @time for n in 1:length(mus1)
        fasta_file = "../res_algos/alpha$(alpha)_potts_seq$(idx)_mu$(round(mus1[n], 
        digits = 2)).fa"
    
        strategy = ASRMethod(;joint = false, # (default) - joint reconstruction not functional yet
            ML = true, # (default)
            verbosity = 1, # the default is 0. 
            optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
            optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
            repetitions = 1, # (default) - for Bayesian reconstruction, multiple repetitions of the reconstruction process can be done to sample likely ancestors
            );
        opt_tree, reconstructed_sequences = infer_ancestral(
            tree_file, fasta_file, ar_model, strategy
            );
        res_ASR_arDCA[idx,:,n] = KitMSA.string2vec(reconstructed_sequences["NODE_1"]);
        node_list=["NODE_1"];
        
        
        #this commented part is for the bayesian arDCA method
        strategy_bayesian = ASRMethod(;
	joint = false, # (default) - joint reconstruction not functional yet
	ML = false, # (default)
	verbosity = 1, # the default is 0. 
	optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
	optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
	repetitions = 100,

)
        
    file_out = "../res_algos/ardca_inf/alpha$(alpha)_ardca_seq$(idx)_mu$(round(mus1[n], 
        digits = 2)).fa";
        infer_ancestral(
               tree_file, fasta_file, ar_model, strategy_bayesian;
               alignment_per_node=true, node_list = ["NODE_1"], outfasta = file_out);
    println(n)
    end 

rec_msas_arDCA = Matrix{Matrix{Int}}(undef, 3, length(mus1));
ens_arDCA = Matrix{Array{Float64,1}}(undef, 3, length(mus1));
hams_arDCA = Matrix{Array{Float64,1}}(undef, 3, length(mus1));
ens_ML_arDCA =  zeros(3, length(mus1));
hams_ML_arDCA =  zeros(3, length(mus1));



    for n in 1:length(mus1)
        
        hams_ML_arDCA[idx,n] = ham_dist_nogap(Int.(start_msa[:,idx]), res_ASR_arDCA[idx,:,n])
        ens_ML_arDCA[idx,n] = PhyloTools.energy(res_ASR_arDCA[idx,:,n], h, J)
    
        rec_msas_arDCA[idx,n] = read_fasta_alignment("../res_algos/ardca_inf/alpha$(alpha)_ardca_seq$(idx)_mu$(round(mus1[n], 
        digits = 2))_NODE_1.fa", 0.9);
        
        hams_arDCA[idx,n] = ham_dist_nogap(Int.(start_msa[:,idx]), Int.(rec_msas_arDCA[idx,n]))
        ens_arDCA[idx,n] = PhyloTools.energy(rec_msas_arDCA[idx,n], h, J)
    end



close("all"); 
    plt.errorbar(mus1, [mean(hams[idx,n])./L for n in 1:length(mus1)],
    yerr = [std(hams[idx,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    plt.scatter(mus1, hams_ML[1,:]./L, marker = "X", color = "blue", label = "Felse ML");
    
plt.errorbar(mus1, [mean(hams_arDCA[idx,n])./L for n in 1:length(mus1)],
    yerr = [std(hams_arDCA[idx,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    plt.scatter(mus1, hams_ML_arDCA[1,:]./L, marker = "X", color = "red", label = "arDCA ML");

plt.xlabel("Mutation rate");
plt.ylabel("Hamming distance");
plt.title("Alpha = $(alpha)");
    plt.legend(); savefig("../prova_seq$(idx)_alpha$(alpha).png")





In [ ]:



idx = 1;mus1 = [1., 10., 50., 100., 200.];

alphas = [0.01, 0.1, 1., 10., 100.];
c = ["blue", "red", "green", "orange", "purple"];


num = 1;
close("all"); 
for alpha in alphas


res_ASR = zeros(Int, 3, L, length(mus1));
pp = Matrix{Matrix{Float64}}(undef, 3, length(mus1));
rec_msas = Matrix{Matrix{Int}}(undef, 3, length(mus1));
#rec_msas_reshuf = Matrix{Matrix{Int}}(undef, 3, length(mus));
ens = Matrix{Array{Float64,1}}(undef, 3, length(mus1));
hams = Matrix{Array{Float64,1}}(undef, 3, length(mus1));
#ens_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
#hams_reshuf = Matrix{Array{Float64,1}}(undef, 3, length(mus));
ens_ML =  zeros(3, length(mus1));
hams_ML =  zeros(3, length(mus1));

    
    for n in 1:length(mus1)
        file_seqs = "../res_algos/alpha$(alpha)_potts_seq$(idx)_mu$(round(mus1[n], 
        digits = 2)).fa"
        mu = infer_mu(tree_file, file_seqs)
        res, p = Felsenstein2(file_seqs, tree_file, file_seqs, mu)
        res_ASR[idx,:,n] .= Int.(res)
        pp[idx,n] = p
        rec_msas[idx,n] = sampling_ANC(pp[idx,n], n_seq = 500)
        hams[idx,n] = ham_dist_nogap(Int.(start_msa[:,idx]), rec_msas[idx,n])
        ens[idx,n] = PhyloTools.energy(rec_msas[idx,n], h, J)
        #@time rec_msas_reshuf[idx,n] = reshuffle_entr(rec_msas[idx,n], h, J, temp = 0.2)
        #hams_reshuf[idx,n] = ham_dist(Int.(start_msa[:,idx]), rec_msas_reshuf[idx,n])
        #ens_reshuf[idx,n] = energy(rec_msas_reshuf[idx,n], h, J)
        hams_ML[idx,n] = ham_dist_nogap(Int.(start_msa[:,idx]), res_ASR[idx,:,n])
        ens_ML[idx,n] = PhyloTools.energy(res_ASR[idx,:,n], h, J)
    end


nat_file = "../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m";


rec_msas_arDCA = Matrix{Matrix{Int}}(undef, 3, length(mus1));
ens_arDCA = Matrix{Array{Float64,1}}(undef, 3, length(mus1));
hams_arDCA = Matrix{Array{Float64,1}}(undef, 3, length(mus1));

    for n in 1:length(mus1)    
        rec_msas_arDCA[idx,n] = read_fasta_alignment("../res_algos/ardca_inf/alpha$(alpha)_ardca_seq$(idx)_mu$(round(mus1[n], 
        digits = 2))_NODE_1.fa", 0.9);
        
        hams_arDCA[idx,n] = ham_dist_nogap(Int.(start_msa[:,idx]), Int.(rec_msas_arDCA[idx,n]))
        ens_arDCA[idx,n] = PhyloTools.energy(rec_msas_arDCA[idx,n], h, J)
    end




    #=plt.errorbar(mus1, [mean(hams[idx,n])./L for n in 1:length(mus1)],
    yerr = [std(hams[idx,n])./L for n in 1:length(mus1)], 
    color = c[num], linewidth = 3.0, label = "Felse alpha = $(alpha)"); =#
    #plt.scatter(mus1, hams_ML[idx,:]./L, marker = "X", color = "blue", label = "Felse ML");
    
plt.errorbar(mus1, [mean(hams_arDCA[idx,n])./L for n in 1:length(mus1)],
    yerr = [std(hams_arDCA[idx,n])./L for n in 1:length(mus1)], 
    color = c[num], alpha = 0.3, label = "arDCA alpha = $(alpha)"); 
    #plt.scatter(mus1, hams_ML_arDCA[idx,:]./L, marker = "X", color = "red", label = "arDCA ML");



num+=1
           
end



plt.xlabel("Mutation rate");
plt.ylabel("Hamming distance");

    plt.legend(); savefig("../prova_seq$(idx)_different_alphas_arDCA.png")
